## Capstone project

- In this assignment, I will be required to explore, segment, and cluster the neighborhoods in the city of Toronto. However, unlike New York, the neighborhood data is not readily available on the internet. What is interesting about the field of data science is that each project can be challenging in its unique way, so I need to learn to be agile and refine the skill to learn new libraries and tools quickly depending on the project.


- For the Toronto neighborhood data, a Wikipedia page exists that has all the information I need to explore and cluster the neighborhoods in Toronto. I will be required to scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format like the New York dataset.

In [1]:
# importing libraries

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
import requests

In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


##### Scrapping the wikipedia page

 - I am going to request the web page through request library and read it's HTML code via beautiful soup library, after getting all the HTML of wikipedia page, now I want specific table from that page, for that I need to find HTML code for that table, I used find method to find table tag inside that HTML and got HTML code for the table.

In [3]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(source, 'lxml')

table = soup.find('table')

#print(table.prettify())

# header = table.th.text

# print(header)

Now, I have data of table in HTML, to convert it into pandas dataframe, I used read_html method of pandas library and pass the HTML of table in it, now I have converted HTML to dataframe

In [4]:
post_code_can = pd.read_html(table.prettify(),header=None,skiprows=0)[0]

Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [5]:
post_code_can = post_code_can[post_code_can.Borough != 'Not assigned']

More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [6]:
df1=post_code_can.groupby("Postcode").agg(lambda x:','.join(set(x)))

In [13]:
df1.head()

,Borough,Neighborhood
Postcode,,
M1B,Scarborough,"Malvern,Rouge"
M1C,Scarborough,"Rouge Hill,Port Union,Highland Creek"
M1E,Scarborough,"Morningside,West Hill,Guildwood"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [14]:
df1.loc[df1['Neighborhood']=="Not assigned",'Neighborhood']=df1.loc[df1['Neighborhood']=="Not assigned",'Borough']

In [15]:
# test

df1.loc[df1.Borough == "Queen's Park"]

,Borough,Neighborhood
Postcode,,
M7A,Queen's Park,Queen's Park


In [16]:
df1

,Borough,Neighborhood
Postcode,,
M1B,Scarborough,"Malvern,Rouge"
M1C,Scarborough,"Rouge Hill,Port Union,Highland Creek"
M1E,Scarborough,"Morningside,West Hill,Guildwood"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
M1J,Scarborough,Scarborough Village
M1K,Scarborough,"Kennedy Park,Ionview,East Birchmount Park"
M1L,Scarborough,"Oakridge,Clairlea,Golden Mile"
M1M,Scarborough,"Cliffside,Cliffcrest,Scarborough Village West"


In [17]:
df1.shape

(103, 2)

### Finding Latitute and Longitude of Postal Code

In [18]:
geo_code = pd.read_csv('https://cocl.us/Geospatial_data')

# geo_data=pd.read_csv("https://cocl.us/Geospatial_data")
# geo_data

In [19]:
geo_code.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [20]:
df1['Latitute'] = geo_code['Latitude'].values
df1['Longitude'] = geo_code['Longitude'].values

In [21]:
df1

,Borough,Neighborhood,Latitute,Longitude
Postcode,,,,
M1B,Scarborough,"Malvern,Rouge",43.806686,-79.194353
M1C,Scarborough,"Rouge Hill,Port Union,Highland Creek",43.784535,-79.160497
M1E,Scarborough,"Morningside,West Hill,Guildwood",43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476
M1J,Scarborough,Scarborough Village,43.744734,-79.239476
M1K,Scarborough,"Kennedy Park,Ionview,East Birchmount Park",43.727929,-79.262029
M1L,Scarborough,"Oakridge,Clairlea,Golden Mile",43.711112,-79.284577
M1M,Scarborough,"Cliffside,Cliffcrest,Scarborough Village West",43.716316,-79.239476


### Part 3

In [22]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [23]:
neighborhoods = df1
df1.head()

,Borough,Neighborhood,Latitute,Longitude
Postcode,,,,
M1B,Scarborough,"Malvern,Rouge",43.806686,-79.194353
M1C,Scarborough,"Rouge Hill,Port Union,Highland Creek",43.784535,-79.160497
M1E,Scarborough,"Morningside,West Hill,Guildwood",43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [24]:
toronto_data= neighborhoods[neighborhoods['Borough'].str.contains('Toronto', na = False)].reset_index(drop=False)
toronto_data.head()

,Postcode,Borough,Neighborhood,Latitute,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"Riverdale,The Danforth West",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [25]:

toronto_data.shape

(39, 5)

##### From google : the latitud and longitud for Toronto, as follows: 43.6532° N, 79.3832° W

In [26]:
latitude = 43.6532
longitude= -79.3832

#### Let's create a map of Toronto with Boroughs that cointain the word Toronto

In [28]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map toronto_data['Latitute']
for lat, lng, label in zip(toronto_data['Latitute'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.
#### Define Foursquare Credentials and Version

In [29]:
CLIENT_ID = 'OSWJ5S2V35IPYD2HQSJPYQATDYW3F04NCP2HBF2MRBVC1VCT'
CLIENT_SECRET = 'TW3KEEK5IG53SUM5CFE5AJNJJSLK1P4IXUXRN41AED5ZFMEK'

### Let's create a function to repeat the same process to all the neighborhoods in Manhattan Lab example

In [30]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
VERSION = 'YYYYMMDD'

In [41]:
def getNearbyVenues(names, latitudes, longitudes, radius=5000):
    LIMIT=100
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)

        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId=56aa371be4b08b9a8d573508,4bf58dd8d48988d1e5941735,5032897c91d4c4b30a586d69,4bf58dd8d48988d100951735'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat,
            lng,
            radius, 
            LIMIT)

  
        # make the GET request
        results = requests.get(url).json()["response"]#['groups'][0]['items']
        if len(results) > 0:
        
            # return only relevant information for each nearby venue
            venues_list.append([(
                name, 
                lat, 
                lng, 
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']) for v in results])

            nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
            #print(nearby_venues)
            nearby_venues.columns = ['Town', 
                      'Town Latitude', 
                      'Town Longitude', 
                      'Venue', 
                      'Venue Latitude', 
                      'Venue Longitude', 
                      'Venue Category']
        #    nearby_venues.groupby('Venue Category').count()
        #    nearby_venues.head()

            return(nearby_venues)


### Now write the code to run the above function on each neighborhood and create a new dataframe called toronto_venues.

In [42]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitute'],
                                   longitudes=toronto_data['Longitude']
                                  )

The Beaches
Riverdale,The Danforth West
The Beaches West,India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Summerhill East,Moore Park
Summerhill West,South Hill,Deer Park,Rathnelly,Forest Hill SE
Rosedale
St. James Town,Cabbagetown
Church and Wellesley
Harbourfront
Garden District,Ryerson
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Toronto Dominion Centre,Design Exchange
Commerce Court,Victoria Hotel
Roselawn
Forest Hill North,Forest Hill West
Yorkville,North Midtown,The Annex
Harbord,University of Toronto
Grange Park,Chinatown,Kensington Market
CN Tower,Island airport,Harbourfront West,Bathurst Quay,Railway Lands,South Niagara,King and Spadina
Stn A PO Boxes 25 The Esplanade
Underground city,First Canadian Place
Christie
Dovercourt Village,Dufferin
Trinity,Little Portugal
Brockton,Exhibition Place,Parkdale Village
High Park,The Junction South
Parkdale,Roncesvalles
Swansea,R

In [52]:
radius = 500
LIMIT = 100

venues = []

for lat, long, post, borough, neighborhood in zip(toronto_data['Latitute'], toronto_data['Longitude'], toronto_data['Postcode'], toronto_data['Borough'], toronto_data['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]#['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [54]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Postcode', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

ValueError: Length mismatch: Expected axis has 0 elements, new values have 9 elements

In [45]:
print(toronto_venues.shape)
toronto_venues.head()

AttributeError: 'NoneType' object has no attribute 'shape'